In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
url = 'https://ucdavisaggies.com/sports/mens-basketball/stats'

page = requests.get(url) #requests object

soup = BeautifulSoup(page.text, 'html')

In [4]:
table = soup.find_all('table')[1]
print(table)

<table accordion-table="" class="sidearm-table collapse-on-medium accordion" sortable-table="">
<caption class="hide">Overall Individual Statistics</caption>
<thead class="sidearm-primary">
<tr>
<th class="text-left" rowspan="2" scope="col">#</th>
<th class="text-hide" rowspan="2" scope="col">Player</th>
<th rowspan="2" scope="col">GP</th>
<th rowspan="2" scope="col">GS</th>
<th colspan="2" scope="colgroup">Minutes</th>
<th colspan="3" scope="colgroup">FG</th>
<th colspan="3" scope="colgroup">3PT</th>
<th colspan="3" scope="colgroup">FT</th>
<th colspan="2" scope="colgroup">Scoring</th>
<th colspan="4" scope="colgroup">Rebounds</th>
<th rowspan="2" scope="col">PF</th>
<th rowspan="2" scope="col">AST</th>
<th rowspan="2" scope="col">TO</th>
<th rowspan="2" scope="col">STL</th>
<th rowspan="2" scope="col">BLK</th>
<th aria-hidden="true" class="hide-on-large" rowspan="2" scope="col">Bio Link</th>
</tr>
<tr>
<th scope="col">TOT</th>
<th scope="col">AVG</th>
<th scope="col">FGM</th>
<th sco

In [6]:
#ISSUE: COLUMNS ARE OUT OF ORDER
#After #, GP, GS it goes out of order
    #It jumps to 

#extract the table headers
headers = table.find_all('th', attrs={"scope": "col"})
print(headers)
num_headers = len(headers)

#loop through the headers
table_headers = [title.text.strip() for title in headers]

#Move these headers to the back of the list to fix the order
headers_to_move = ["PF", "AST", "AST/G", "TO", "STL", "BLK", "Bio Link"]

fixed_headers = [h for h in table_headers if h not in headers_to_move]  # Keep other headers in original order
moved_headers = [h for h in table_headers if h in headers_to_move]  # Move these to the end

# Combine lists to get the final order
final_headers = fixed_headers + moved_headers
print(final_headers)


[<th class="text-left" rowspan="2" scope="col">#</th>, <th class="text-hide" rowspan="2" scope="col">Player</th>, <th rowspan="2" scope="col">GP</th>, <th rowspan="2" scope="col">GS</th>, <th rowspan="2" scope="col">PF</th>, <th rowspan="2" scope="col">AST</th>, <th rowspan="2" scope="col">TO</th>, <th rowspan="2" scope="col">STL</th>, <th rowspan="2" scope="col">BLK</th>, <th aria-hidden="true" class="hide-on-large" rowspan="2" scope="col">Bio Link</th>, <th scope="col">TOT</th>, <th scope="col">AVG</th>, <th scope="col">FGM</th>, <th scope="col">FGA</th>, <th scope="col">FG%</th>, <th scope="col">3PT</th>, <th scope="col">3PTA</th>, <th scope="col">3PT%</th>, <th scope="col">FTM</th>, <th scope="col">FTA</th>, <th scope="col">FT%</th>, <th scope="col">PTS</th>, <th scope="col">AVG</th>, <th scope="col">OFF</th>, <th scope="col">DEF</th>, <th scope="col">TOT</th>, <th scope="col">AVG</th>]
['#', 'Player', 'GP', 'GS', 'TOT', 'AVG', 'FGM', 'FGA', 'FG%', '3PT', '3PTA', '3PT%', 'FTM', 'FT

In [7]:
#Extract data for columns
column_data = table.find_all('tr')
df = pd.DataFrame(columns = final_headers)

for row in column_data[2:]: 
    row_data = row.find_all('td')
    individual_row_data = [data.get_text(separator = " ").strip() for data in row_data]
    individual_row_data[1] = " ".join(individual_row_data[1].split()).split(" ", 1)[-1]
    if len(individual_row_data) > 1:
        words = individual_row_data[1].split()  # Split into words
    if len(words) > 2:  
        individual_row_data[1] = " ".join(words[2:])  # Keep everything after the first two words
        
    print(individual_row_data)
    
    length = len(df)
    df.loc[length] = individual_row_data




['02', 'Johnson, TY', '26', '26', '866', '33.3', '185', '476', '.389', '27', '105', '.257', '159', '203', '.783', '556', '21.4', '25', '94', '119', '4.6', '61', '67', '102', '61', '4', 'View Bio']
['08', 'Tamba, Pablo', '26', '26', '699', '26.9', '109', '178', '.612', '3', '14', '.214', '58', '92', '.630', '279', '10.7', '35', '95', '130', '5.0', '69', '28', '59', '28', '21', 'View Bio']
['04', 'Sevilla, Connor', '26', '18', '735', '28.3', '74', '191', '.387', '51', '137', '.372', '35', '36', '.972', '234', '9.0', '12', '28', '40', '1.5', '42', '24', '22', '12', '0', 'View Bio']
['10', 'DeBruhl, Leo', '26', '26', '748', '28.8', '69', '197', '.350', '32', '91', '.352', '51', '60', '.850', '221', '8.5', '22', '79', '101', '3.9', '62', '79', '63', '37', '1', 'View Bio']
['07', 'Rocak, Niko', '26', '23', '663', '25.5', '48', '92', '.522', '9', '25', '.360', '42', '59', '.712', '147', '5.7', '59', '131', '190', '7.3', '79', '38', '40', '22', '54', 'View Bio']
['01', '01 Daughtery Jr., Carl'

In [9]:
df

#Export this into a csv
df.to_csv(r'/Users/ryanuyeki/Downloads/asa/ucd_player_stats.csv', index=False, encoding = "utf-8")